### well structured - an ode to databases


# data preparation

### import libraries

In [510]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

## build stock preparation class

In [511]:
class stock_prep:
    def __init__(self, company_name):

        # load data
        self.company_stock_prices = pd.read_csv('data/' + company_name + '.csv')[['Date', 'Close']] #input genauere Beschreibung
        self.company_name = company_name

        # return dfs
        self.replace_company_stock_prices = []
        self.avg_norm_loss_company_stock_prices = []
        self.formatted_date_to_index = []
        self.standardization_company_stock_prices = []
        self.normalization_company_stock_pricecs = []
        self.binning_company_stock_prices = []
        self.indicator_variables_stock_prices = []

    def pipeline_conductor(self,pipeline):
        pass

    def replace_values(self):
        print(self.company_stock_prices.head(5))
        self.replace_company_stock_prices = self.company_stock_prices.replace("?", np.nan)
        print(self.replace_company_stock_prices.head(5))

    def missing_values_count(self):
        self.missing_company_stock_prices = self.replace_company_stock_prices.isnull()
        print(self.missing_company_stock_prices.head(5))

        for column in self.missing_company_stock_prices.columns.values.tolist():
            print(column)
            print (self.missing_company_stock_prices[column].value_counts()) 

    def avg_norm_loss(self):
        avg_norm_loss = self.replace_company_stock_prices["Close"].astype("float").mean(axis=0)
        print("Average of normalized-losses:", avg_norm_loss)
        print(self.replace_company_stock_prices['Date'])
        self.avg_norm_loss_company_stock_prices = pd.concat([self.replace_company_stock_prices['Date'], self.replace_company_stock_prices['Close'].replace(np.nan, avg_norm_loss)], axis=1)
        print(self.avg_norm_loss_company_stock_prices.head(5))

    def format_date_to_index(self):
        self.formatted_date_to_index = pd.concat([self.avg_norm_loss_company_stock_prices['Date'].str.replace('-', '', regex=True).astype(np.int64), self.avg_norm_loss_company_stock_prices['Close'].astype(np.float64)], axis=1)
        print(self.formatted_date_to_index)
        print(self.formatted_date_to_index.dtypes)

    def standardization(self):
        # currency dollar euro
        self.standardization_company_stock_prices = pd.concat([self.formatted_date_to_index['Date'], (self.formatted_date_to_index['Date']/self.formatted_date_to_index["Close"]).rename("Close")], axis=1)
        print(self.standardization_company_stock_prices)
        
    def normalization(self):
        # replace (original value) by (original value)/(maximum value)
        self.normalization_company_stock_pricecs['Close'] = self.standardization_company_stock_prices['Close']/self.standardization_company_stock_prices['Close'].max()
        
    def binning(self):
        # build bins
        self.binning_company_stock_prices = np.linspace(min(self.normalization_company_stock_pricecs["horsepower"]), max(self.normalization_company_stock_pricecs["horsepower"]), 4)

    def plot_bins(self):

        plt.pyplot.hist(self.normalization_company_stock_pricecs["Close"], bins = 3)
       
        plt.pyplot.xlabel("Date")
        plt.pyplot.ylabel("Close")
        plt.pyplot.title("Close_Price")

    #def indicator_variables(self):
        #dummy_variable_1 = pd.get_dummies(self.binning_company_stock_prices["Close"])
        #dummy_variable_1.head()
        #dummy_variable_1.rename(columns={'Date':'fuel-type-gas', 'diesel':'fuel-type-diesel'}, inplace=True)
        #dummy_variable_1.head()
        # merge data frame "df" and "dummy_variable_1" 
        #df = pd.concat([df, dummy_variable_1], axis=1)

        #drop original column "fuel-type" from "df"
        #df.drop("fuel-type", axis = 1, inplace=True)

    def to_sqlite_database(self):
        conn = sqlite3.connect('stocks_prep.db')
        self.data['Close'].to_sql(name=self.company_name, con=conn, if_exists='replace')
        conn.commit()
        conn.close() 

In [512]:
stock_prep_obj = stock_prep('iShares Global Timber & Forestry UCITS ETF USD Acc')

## identify and handling missing values

In [513]:
#replace missing values
stock_prep_obj.replace_values()

         Date               Close
0  2021-11-05                   ?
1  2021-11-08   30.70560073852539
2  2021-11-09  30.667200088500977
3  2021-11-10   30.44179916381836
4  2021-11-11  30.740800857543945
         Date               Close
0  2021-11-05                 NaN
1  2021-11-08   30.70560073852539
2  2021-11-09  30.667200088500977
3  2021-11-10   30.44179916381836
4  2021-11-11  30.740800857543945


In [514]:
#identify missing values
stock_prep_obj.missing_values_count()

    Date  Close
0  False   True
1  False  False
2  False  False
3  False  False
4  False  False
Date
Date
False    609
Name: count, dtype: int64
Close
Close
False    608
True       1
Name: count, dtype: int64


In [515]:
#handling missing values
stock_prep_obj.avg_norm_loss()

Average of normalized-losses: 26.66417549472106
0      2021-11-05
1      2021-11-08
2      2021-11-09
3      2021-11-10
4      2021-11-11
          ...    
604    2024-03-22
605    2024-03-25
606    2024-03-26
607    2024-03-27
608    2024-03-28
Name: Date, Length: 609, dtype: object
         Date               Close
0  2021-11-05           26.664175
1  2021-11-08   30.70560073852539
2  2021-11-09  30.667200088500977
3  2021-11-10   30.44179916381836
4  2021-11-11  30.740800857543945


## correct dataformat

In [516]:
stock_prep_obj.format_date_to_index()

         Date      Close
0    20211105  26.664175
1    20211108  30.705601
2    20211109  30.667200
3    20211110  30.441799
4    20211111  30.740801
..        ...        ...
604  20240322  28.428900
605  20240325  28.478600
606  20240326  28.685301
607  20240327  28.638100
608  20240328  28.737900

[609 rows x 2 columns]
Date       int64
Close    float64
dtype: object


## data standardization

In [517]:
stock_prep_obj.standardization()


         Date          Close
0    20211105  757987.247871
1    20211108  658222.197706
2    20211109  659046.438595
3    20211110  663926.264385
4    20211111  657468.590154
..        ...            ...
604  20240322  711962.902796
605  20240325  710720.517196
606  20240326  705599.223869
607  20240327  706762.223504
608  20240328  704307.835811

[609 rows x 2 columns]


## binning

In [518]:
stock_prep_obj.binning()

TypeError: list indices must be integers or slices, not str

## indicator Variables

In [ ]:
stock_prep_obj.indicator_variables()

# store data in sqlite db

In [ ]:
ticker_symbols = ['WOOE.AS', 'WY', 'SCA-B.ST', 'SUZ', 'SK3.IR', 'IP', 'STERV.HE','RYN', 'WFG', 'WRK', 'PCH']
os.listdir(path)

for i in range(len(ticker_symbols)): 
    stock_obj = stock_prep(ticker_symbols[i])
    #identify missing values
    stock_prep_obj.missing_values_count()

    #handling missing values
    stock_prep_obj.avg_norm_loss()

    #handling normalized loses
    stock_prep_obj.normalized_loses()

    #data_format
    stock_prep_obj.data_format()

    #data standardization
    stock_prep_obj.standardization()

    #binning
    stock_prep_obj.binning()

    #indicator variables
    stock_prep_obj.indicator_variables()

    #data in sqlite database
    stock_prep_obj.to_sqlite_database()
